In [7]:
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Number of pages to scrape
num_pages = 10

# Delay between requests (in seconds)
delay = 1

# Initialize an empty list to store all game links
all_links = []

# Iterate over each page
for page in range(41, num_pages + 41):
    # Construct the URL for the current page
    url = f"https://store.steampowered.com/search/?sort_by=Reviews_DESC&force_infinite=1&category1=998&supportedlang=english&snr=1_7_7_230_7&page={page}"
    
    # Make a request to the current page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract individual game links on the search page
    result_rows_div = soup.find('div', id='search_resultsRows')
    a_elements = result_rows_div.find_all('a')
    
    # Extract the links and append them to the all_links list
    links_list = [a_element['href'] for a_element in a_elements]
    all_links.extend(links_list)
    
    # Introduce a delay between requests
    time.sleep(delay)

# Scrape game data from each link
game_data = []
for link in all_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    try:
        game_title = soup.find('div', {'class': 'apphub_AppName'}).get_text(strip=True)
    except AttributeError:
        game_title = 'N/A'  # Handle the case when the title is not found

    try:
        game_description = soup.find('div', {'class': 'game_description_snippet'}).get_text(strip=True)
    except AttributeError:
        game_description = 'N/A'
        
    try:
        genre_div = soup.find('div', id='genresAndManufacturer')
        first_span = genre_div.find('span')
        
        game_genre_list = []
        if first_span:
            a_elements_in_span = first_span.find_all('a')
            for a_element in a_elements_in_span:
                genre = a_element.text
                game_genre_list.append(genre)
    except AttributeError:
        game_genre_list = []

    game_data.append({'Title': game_title, 'Description': game_description, 'Genre': game_genre_list})

# Create a DataFrame from the scraped game data
df = pd.DataFrame(game_data)

# Save the DataFrame to a CSV file
df.to_csv('steam_games.csv', mode='a', header=False, index=False)
